In [1]:
import pyidi
import importlib
import os
import dill
import pickle
import numpy as np
import matplotlib.pyplot as plt 
import math
import re
import pandas as pd
import warnings
import matplotlib.pyplot as plt
from mpl_interactions import zoom_factory
# import matplotlib.image as mpimg
warnings.filterwarnings('ignore')


c:\Users\thijs\anaconda3\lib\site-packages\skimage\util\dtype.py:27: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  np.bool8: (False, True),


In [2]:
# folder_paths = ['D:/HSC', 'F:/', 'C:/Users/thijs/Documents/HSC/', 'H:/My Drive/PHD/HSC']  #, 'H:/My Drive/PHD/HSC'
# sequential_image_n = 0
# file_names = []
# # loop video files
# fix, ax = plt.subplots(1, 3, figsize=(25, 6))
# plt.tight_layout()

# tested_locations = [[], [], []]
# df = pd.DataFrame(columns=['filename', 'ecc', 'loc','mass', 'taut/loose', 'impact/pluck'])
# still_images_mean = np.zeros((3, 512, 1024))
# for folder_path in folder_paths:
#     for root, dirs, files in os.walk(os.path.normpath(folder_path), topdown=False):
#         for file in files:
#             if not file.endswith('.mraw') or not file.startswith('Full_web_') or file.startswith('Full_web_ecc7'):
#                 continue
#             sections = file.split('_')
#             if 'nomass' in sections or 'taut' in sections or 'loose' in sections:
#                 continue
#             file = os.path.splitext(file)[0] + '.cihx'
#             if not file in file_names:
#                 for section in sections:
#                     if section.startswith('ecc'):
#                         n_ax = int(section[-1])
#                     elif section.startswith('Floc'):
#                         n_loc = int(re.findall('\d+', section)[0])
#                 file_names.append(file)
#                 if n_loc not in tested_locations[n_ax]:
#                     tested_locations[n_ax].append(n_loc)
#                     file_path = os.path.join(root, file)
#                     video = pyidi.pyIDI(file_path)
#                     still_image_new = video.mraw[sequential_image_n]
#                     n_images = len(tested_locations[n_ax])
#                     still_images_mean[n_ax] = still_images_mean[n_ax]*(n_images-1)/n_images
#                     still_images_mean[n_ax] += still_image_new*(1/n_images)
#                 n_ax = np.nan
#                 n_loc = np.nan

# for n_ax in range(3):
#     ax[n_ax].imshow(still_images_mean[n_ax], cmap='gray')
# plt.show()
# len(file_names), file_names, tested_locations

In [23]:
folder_paths = ['D:/HSC', 'F:/', 'C:/Users/thijs/Documents/HSC/', 'H:/My Drive/PHD/HSC']  #, 'H:/My Drive/PHD/HSC'
file_names = []
folder_paths = ['D:/HSC', 'F:/', 'C:/Users/thijs/Documents/HSC/', 'H:/My Drive/PHD/HSC']  #, 'H:/My Drive/PHD/HSC'
df = pd.DataFrame(columns=['filename', 'path', 'ecc', 'loc','mass', 'taut/loose', 'impact/pluck', 'version', 'n_frames', 'fps', 'spider_ij', 'prey_ij'])
for folder_path in folder_paths:
    for root, dirs, files in os.walk(os.path.normpath(folder_path), topdown=False):
        for file in files:
            if not file.endswith('.mraw') or not file.startswith('Full_web_') or file.startswith('Full_web_ecc7'):
                continue
            file = os.path.splitext(file)[0] + '.cihx'
            if file in file_names:
                continue   
            file_names.append(file)
        
            sections = file.split('_')
            version = 0
            for section in sections:
                if section.startswith('ecc'):
                    n_ax = int(section[-1])
                elif section.startswith('Floc'):
                    n_loc = int(re.findall('\d+', section)[0])
                elif section.startswith('v'):
                    version = int(section[1:])

            if 'nomass' in sections:
                mass = False
            else:
                mass = True
            if 'taut' in sections:
                taut_loose = 'Taut'
            elif 'loose' in sections:
                taut_loose = 'Loose'
            else:
                taut_loose = 'Loose'
            if 'impact' in sections:
                impact_pluck = 'Impact'
            elif 'pluck' in sections:
                impact_pluck = 'Pluck'
            else:
                impact_pluck = 'Impact'
            
            file_path = os.path.join(root, file)
            try:
                video = pyidi.pyIDI(file_path)
            except:
                print('Could not open file: ' + file_path)
                continue
            n_frames = video.info['Total Frame']
            fps = video.info['Record Rate(fps)']
            video.close_video()

            df = df.append({'filename': file,
                            'path': file_path,
                            'ecc': n_ax, 
                            'loc': n_loc,
                            'mass': mass,
                            'taut/loose': taut_loose, 
                            'impact/pluck': impact_pluck, 
                            'version': version, 
                            'n_frames': n_frames,
                            'fps': fps,
                            'spider_ij': (0,0),
                            'prey_ij': (0,0)}, ignore_index=True)
            del n_ax, n_loc, mass, taut_loose, impact_pluck, file_path

df.to_csv('H:/My Drive/PHD/HSC/file_descriptions.csv', index=False)

Could not open file: H:\My Drive\PHD\HSC\ecc2\web1\taut impact\Full_web_ecc2_new2_Floc15_taut_impact_v0_S01\Full_web_ecc2_new2_Floc15_taut_impact_v0_S01.cihx
Could not open file: H:\My Drive\PHD\HSC\ecc2\web1\taut pluck\Full_web_ecc2_new2_Floc14_taut_pluck_v0_S01\Full_web_ecc2_new2_Floc14_taut_pluck_v0_S01.cihx
Could not open file: H:\My Drive\PHD\HSC\ecc2\web1\loose pluck\Full_web_ecc2_new2_Floc19_loose_pluck_v0_S01\Full_web_ecc2_new2_Floc19_loose_pluck_v0_S01.cihx


In [20]:
def break_event(event):
        """check which points are inside the polygon when the user presses 'enter'
        The points inside the selection are deleted from the ROI
        """
        global _STOP
        if event.key == 'c':
            plt.close()
            _STOP = True
        

df = pd.read_csv('D:/HSC/file_descriptions.csv')
%matplotlib qt
sequential_image_n = 0
_STOP = False
spider_ij = []
prey_ij = []
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
zoom_factory(ax)
fig.canvas.mpl_connect('key_press_event', break_event)
text = ax.text(0, 5, '', ha='left', va='center', fontdict={'size': 20})
for id, path_temp in enumerate(df['path']):
    if df['prey_ij'][id] != '(0, 0)':
        continue
    video = pyidi.pyIDI(path_temp)
    still_image = video.mraw[sequential_image_n]
    cf = ax.imshow(still_image, cmap='gray')
    plt.title(df['filename'][id])
    # text.set_text('click the spider')
    # x, y = plt.ginput(1)[0]
    # df['spider_ij'][id] = (int(x), int(y))
    text.set_text('click the prey')
    x, y = plt.ginput(1)[0]
    df['prey_ij'][id] = (int(x), int(y))
    del cf
    if _STOP:
        break
plt.close()


In [22]:
df.to_csv('H:/My Drive/PHD/HSC/file_descriptions_with_locations.csv', index=False)

In [10]:
df

,filename,path,ecc,loc,mass,taut/loose,impact/pluck,version,n_frames,fps,spider_ij,prey_ij
0,Full_web_ecc0_new_Floc1_v0_S01.cihx,D:\HSC\ecc0\web0\Full_web_ecc0_new_Floc1_v0_S0...,0,1,True,Loose,Impact,0,10000,8000.0,"(533, 215)","(0, 0)"
1,Full_web_ecc0_new_Floc10_v1_S01.cihx,D:\HSC\ecc0\web0\Full_web_ecc0_new_Floc10_v1_S...,0,10,True,Loose,Impact,1,10000,8000.0,"(533, 215)","(0, 0)"
2,Full_web_ecc0_new_Floc10_v0_S01.cihx,D:\HSC\ecc0\web0\Full_web_ecc0_new_Floc10_v0_S...,0,10,True,Loose,Impact,0,10001,8000.0,"(531, 272)","(0, 0)"
3,Full_web_ecc0_new_Floc9_v1_S01.cihx,D:\HSC\ecc0\web0\Full_web_ecc0_new_Floc9_v1_S0...,0,9,True,Loose,Impact,1,10000,8000.0,"(528, 273)","(0, 0)"
4,Full_web_ecc0_new_Floc9_v0_S01.cihx,D:\HSC\ecc0\web0\Full_web_ecc0_new_Floc9_v0_S0...,0,9,True,Loose,Impact,0,10000,8000.0,"(528, 273)","(0, 0)"
...,...,...,...,...,...,...,...,...,...,...,...,...
213,Full_web_ecc2_new2_Floc14_loose_pluck_v0_S01.cihx,H:\My Drive\PHD\HSC\ecc2\web1\loose pluck\Full...,2,14,True,Loose,Pluck,0,8000,8000.0,"(0, 0)","(0, 0)"
214,Full_web_ecc2_new2_Floc15_loose_pluck_v0_S01.cihx,H:\My Drive\PHD\HSC\ecc2\web1\loose pluck\Full...,2,15,True,Loose,Pluck,0,8000,8000.0,"(0, 0)","(0, 0)"
215,Full_web_ecc2_new2_Floc16_loose_pluck_v0_S01.cihx,H:\My Drive\PHD\HSC\ecc2\web1\loose pluck\Full...,2,16,True,Loose,Pluck,0,8000,8000.0,"(0, 0)","(0, 0)"
216,Full_web_ecc2_new2_Floc17_loose_pluck_v0_S01.cihx,H:\My Drive\PHD\HSC\ecc2\web1\loose pluck\Full...,2,17,True,Loose,Pluck,0,8000,8000.0,"(0, 0)","(0, 0)"
